In [17]:
%cd C://Users/ethan/Documents/Github/Purdue-PHYS-580/Labs/Lab6

C:\Users\ethan\Documents\Github\Purdue-PHYS-580\Labs\Lab6


In [29]:
# !!dir

In [30]:
%pylab --no-import-all

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [31]:
# %load -s Body Body.py
class Body:
    def __init__(self, mass, body, mass_scale=1.0):  # Default scaling gives back units of AU and seconds
        self.vecs = body.vectors()
        ## Manually Convert AU to m for further scaling
        self.pos = np.asarray([self.vecs['x'][0], self.vecs['y'][0], self.vecs['z'][0]])
        self.vel = np.asarray([self.vecs['vx'][0], self.vecs['vy'][0], self.vecs['vz'][0]])
        self.mass = mass / mass_scale

In [2]:
from sympy import symbols, Matrix, lambdify
from sympy.physics.mechanics import Particle, Point, dynamicsymbols, Lagrangian, LagrangesMethod, ReferenceFrame

from scipy.integrate import solve_ivp
from functools import partial
from astroquery.jplhorizons import Horizons
from numpy import apply_along_axis as thread
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(sign=' ', linewidth=100, precision=4, suppress=True)
plt.style.use('dark_background')

In [3]:
# Gravitational Constant In Our Units
day_s = 86400.0
AU_m = 149597900000.0

sun_mass_kg = 1.9891e30
earth_mass_kg = 5.97e24

mass_unit = sun_mass_kg
length_unit = AU_m
time_unit = day_s

Gravity = 6.674e-11 * mass_unit * time_unit ** 2 * length_unit ** (-3) #39.44
print(f'Gravitational Constant In Our Units: {Gravity}')

duration = 365.0
fps = 1200.0
t_eval = np.linspace(0.0, duration, num=int(fps*duration))

Gravitational Constant In Our Units: 0.00029600143285188574


In [5]:
q = dynamicsymbols('q:6')
u = dynamicsymbols('q:6', level=1)
m = symbols('m:2')
G, t = symbols('G t')

In [6]:
O = ReferenceFrame('O')
Home = Point('Home')
Home.set_vel(O, 0)

In [7]:
P0 = Point(r'P_0')
P1 = Point(r'P_1')

In [8]:
P0.set_pos(Home, q[0] * O.x + q[1] * O.y + q[2] * O.z)    
P1.set_pos(Home, q[3] * O.x + q[4] * O.y + q[5] * O.z)

In [9]:
P0.set_vel(O, u[0] * O.x + u[1] * O.y + u[2] * O.z)
P1.set_vel(O, u[3] * O.x + u[4] * O.y + u[5] * O.z)

In [10]:
p0 = Particle('p_0', P0, m[0])
p1 = Particle('p_1', P1, m[1])

In [11]:
# p0.potential_energy = -G * (p1.mass / P0.pos_from(P1).magnitude())
# p1.potential_energy = -G * (p0.mass / P1.pos_from(P0).magnitude())

p0_force = (P0, -G * (p1.mass * P0.pos_from(P1) / P0.pos_from(P1).magnitude() ** 3))
p1_force = (P1, -G * (p0.mass * P1.pos_from(P0) / P1.pos_from(P0).magnitude() ** 3))
forces = [p0_force, p1_force]
# forces = []

In [12]:
L = Lagrangian(O, p0, p1)

In [13]:
LM = LagrangesMethod(L, q, frame=O, forcelist=forces)

In [14]:
EL = LM.form_lagranges_equations()

In [15]:
rhs = LM.rhs()

In [16]:
earth_info    = Horizons(id = '399', id_type = 'majorbody')
earth_body  = Body( earth_mass_kg,  earth_info, mass_scale=mass_unit, time_scale=time_unit)

NameError: name 'Body' is not defined

In [ ]:
# print(f'Saturn: {saturn_body.pos} | {saturn_body.vel}')
# print(f'Mars:   {mars_body.pos} | {mars_body.vel}')
print(f'Earth:  {earth_body.pos} | {earth_body.vel}')

In [ ]:
val_dict = {G: Gravity, m[0]: 1.0, m[1]: earth_body.mass}
rhs = rhs.subs(val_dict)

df_lambdify = lambdify(Matrix([q, u]), rhs)
df = lambda t, x: df_lambdify(*x).reshape(12)

In [ ]:
state0 = np.hstack((
    np.asarray([0.0,0.0,0.0]), earth_body.pos, 
    np.asarray([0.0,0.0,0.0]), earth_body.vel 
))

In [ ]:
sol = solve_ivp(df, [0.0, duration], state0, t_eval=t_eval, rtol=1.0e-6, atol=1.0e-6)
if sol.success is not True:
    print(sol.message)

In [ ]:
head = 10
for row in sol.y.T[-head:, :]:
    print(row[:3], row[3:6], ' | ', row[6:9], row[9:12])

In [ ]:
p0_pos, p1_pos, p0_vel, p1_vel = np.vsplit(sol.y, 4)
t = sol.t

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111, projection='3d')

ax.plot(p1_pos[0,:] - p0_pos[0,:], p1_pos[1,:] - p0_pos[1,:], p1_pos[2,:] - p0_pos[2,:], label=r'$P_1$')
# ax.plot(p0_pos[0,:], p0_pos[1,:], p0_pos[2,:], label=r'$P_0$')
# ax.plot(p1_pos[0,:], p1_pos[1,:], p1_pos[2,:], label=r'$P_1$')

plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,16))

ax1 = fig.add_subplot(121)
ax1.plot(t, p0_pos[0,:], label=r'$P_0$ x')
ax1.plot(t, p0_pos[1,:], label=r'$P_0$ y')
ax1.plot(t, p0_pos[2,:], label=r'$P_0$ z')
ax1.grid()
ax1.legend()

ax2 = fig.add_subplot(122)
ax2.plot(t, p1_pos[0,:], label=r'$P_1$ x')
ax2.plot(t, p1_pos[1,:], label=r'$P_1$ y')
ax2.plot(t, p1_pos[2,:], label=r'$P_1$ z')
ax2.grid()
ax2.legend()


plt.show()

In [ ]:
fig = plt.figure(figsize=(16,16))

ax1 = fig.add_subplot(121)
ax1.plot(t, np.apply_along_axis(np.linalg.norm, 0, p0_pos), label=r'$P_0$')
ax1.grid()
ax1.legend()

ax2 = fig.add_subplot(122)
ax2.plot(t, np.apply_along_axis(np.linalg.norm, 0, p1_pos), label=r'$P_1$')
ax2.grid()
ax2.legend()


plt.show()